In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os

In [4]:
allFiles = sorted(glob.glob("*.txt")) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
allFiles1 = allFiles[:][5]
allFiles2 = allFiles[:][6:]
print(allFiles)

['SurfaceSHG1.txt', 'SurfaceSHG2.txt', 'SurfaceSHG3.txt', 'SurfaceSHG4.txt', 'SurfaceSHG5.txt', 'background1.txt', 'background2.txt', 'background3.txt', 'background4.txt', 'background5.txt']


In [5]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [6]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [9]:
c = 3*10**11; #Speed of Light in mm/s, Going to use this to convert spatial information to time. 
T1 = (499*0.000125 + 0.0229)*2*25.4/(10**-12*c)
def ConvertToTimeDelay(x,itr,offset):
    c = 3*10**11
    timeDelay = []
    for i in range(0,len(x)):
        timedelay = ((x[i]*itr + offset)*25.4*2/(10**-12*c))-T1
        timeDelay.append(timedelay)
    return(timeDelay)

In [10]:
times = pd.Series()
PDvalues = pd.Series()
itr = 0.000125
offset = 0.023
#print(frame)
for i in range(0,400):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
    
times = times.values.reshape(400,len(allFiles))
PDvalues = PDvalues.values.reshape(400,len(allFiles)) # Reshapes the dataset so that it is (900,?) to average each row
timeDelay = ConvertToTimeDelay(times,itr,offset) #Converts Datapoint number to timeDelay
PDvalues = PDvalues.astype(np.float) #Ensures all the values are floats so that I can plot them!
AveragedSHG = PDvalues[:,:5].mean(axis=1)
AveragedBackground = PDvalues[:,5:].mean(axis=1)

In [59]:
#extraticks = [-10,-9,-8,-7,-6,-5,-4, -3, -2, -1, 1, 2, 3, 4,5,6,7,8,9,10]
plt.plot(times[:,1],AveragedSHG,'b.',label = 'SHG')
plt.plot(times[:,1],AveragedBackground,'r.',label = 'Background')
plt.xlabel('Data Point Number')
plt.ylabel('PD Signal (arb)')
plt.grid(True)
#plt.xticks(list(plt.xticks()[0]) + extraticks)
#plt.xlim(-11,11)
plt.legend(loc = 'upper center')
#plt.show()
plt.savefig('SHG_Measurement_Ag_Mirror.png')